# Pandas

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

2. Рассчитайте среднее значение показателей для каждого из столбцов c номерами 3-6.

3. Добавьте столбец, содержащий только число месяца, к которому относится дата.

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

## Лабораторная работа №2

### Базовые операции с `DataFrame`

In [2]:
import numpy as np
import pandas as pd

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами.

In [18]:
recipes = pd.read_csv("./data/data/recipes_sample.csv")
reviews = pd.read_csv("./data/data/reviews_sample.csv")

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [28]:
f"recipes: {recipes.shape} ", recipes.dtypes, f"reviews:\n{reviews.shape}", reviews.dtypes

('recipes: (30000, 8) ',
 name               object
 id                  int64
 minutes             int64
 contributor_id      int64
 submitted          object
 n_steps           float64
 description        object
 n_ingredients     float64
 dtype: object,
 'reviews:\n(126696, 6)',
 Unnamed: 0     int64
 user_id        int64
 recipe_id      int64
 date          object
 rating         int64
 review        object
 dtype: object)

1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [82]:
recipes.isna().sum(), reviews.isna().sum()

(name                  0
 id                    0
 minutes               0
 contributor_id        0
 submitted             0
 n_steps           11190
 description         623
 n_ingredients      8880
 dtype: int64,
 Unnamed: 0     0
 user_id        0
 recipe_id      0
 date           0
 rating         0
 review        17
 dtype: int64)

In [89]:
(recipes.isna().sum().sum() + reviews.isna().sum().sum()) / (len(recipes.index) + len(reviews.index))

0.1321667432480727

1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

1.5 Создайте серию из 10 случайных названий рецептов.

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами).

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы?

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

### Объединение таблиц `pd.DataFrame`

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты без отзывов должны отсутствовать в данной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и выведя на экран строку из полученного `DataFrame`, содержащую информацию об этом отзыве.

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`. У рецептов, для которых отсутствуют отзывы, в соответствущем столбце должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и выведя на экран строку из полученного `DataFrame`, содержащую информацию об этом отзыве.


5.3. Выясните, отзывы, добавленные в каком году, имеют наименьший средний рейтинг?

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.